**Configuring Logging:**

Python’s standard logging module is used

create_engine.echo and create_engine.echo_pool present on create_engine() which allow *immediate logging* to sys.stdout for the purposes of local development

logging.getLogger('sqlalchemy')

- sqlalchemy.engine - controls *SQL echoing*. echo=True , echo="debug" => Set to logging.INFO for SQL query output, logging.DEBUG for query + result set output.

- sqlalchemy.pool - controls connection *pool logging*.  pool_echo=True , pool_echo="debug"

- sqlalchemy.dialects

- sqlalchemy.orm 

For example, to log SQL queries using Python logging instead of the echo=True flag:

In [1]:
import logging

logging.basicConfig()
logging.getLogger("sqlalchemy.engine").setLevel(logging.INFO)

*More on the Echo Flag:*

In [7]:
import logging 
from sqlalchemy import create_engine, URL, String, select 
from sqlalchemy import Table, Column, MetaData 

e = create_engine("sqlite://", echo=True, echo_pool="debug")

# equivalent

logging.basicConfig()
logging.getLogger("sqlalchemy.engine").setLevel(logging.INFO)
logging.getLogger("sqlalchemy.pool").setLevel(logging.DEBUG)


url = URL.create(
    drivername="postgresql",
    username="postgres",
    host="192.168.0.8",
    port="5432",
    password="1234",
    database="tests"
)

engine = create_engine(url)
meta = MetaData()
t1 = Table("t1", meta, Column("name", String(50), primary_key=True))


with engine.connect() as conn:
    logging.warning("Warning")
    conn.execute(select(t1).where(t1.c.name == "some name 1"))


DEBUG:sqlalchemy.pool.impl.QueuePool:Created new connection <connection object at 0x7f593d8a2c00; dsn: 'user=postgres password=xxx dbname=tests host=192.168.0.8 port=5432', closed: 0>


2023-09-02 13:31:03,525 INFO sqlalchemy.engine.Engine select pg_catalog.version()


INFO:sqlalchemy.engine.Engine:select pg_catalog.version()


2023-09-02 13:31:03,534 INFO sqlalchemy.engine.Engine [raw sql] {}


INFO:sqlalchemy.engine.Engine:[raw sql] {}


...

INFO:sqlalchemy.engine.Engine:SELECT t1.name 
FROM t1 
WHERE t1.name = %(name_1)s


2023-09-02 13:31:03,573 INFO sqlalchemy.engine.Engine [generated in 0.00650s] {'name_1': 'some name 1'}


INFO:sqlalchemy.engine.Engine:[generated in 0.00650s] {'name_1': 'some name 1'}


2023-09-02 13:31:03,582 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK
DEBUG:sqlalchemy.pool.impl.QueuePool:Connection <connection object at 0x7f593d8a2c00; dsn: 'user=postgres password=xxx dbname=tests host=192.168.0.8 port=5432', closed: 0> being returned to pool
DEBUG:sqlalchemy.pool.impl.QueuePool:Connection <connection object at 0x7f593d8a2c00; dsn: 'user=postgres password=xxx dbname=tests host=192.168.0.8 port=5432', closed: 0> reset, transaction already reset


  It’s important to note that these *two flags work independently* of any existing logging configuration, and will make use of logging.basicConfig() *unconditionally*. This has the effect of being configured in addition to any existing logger configurations. Therefore, when configuring logging explicitly, *ensure all echo flags are set to False* at all times, to avoid getting duplicate log lines.

*Setting the Logging Name:*

  use the create_engine.logging_name and create_engine.pool_logging_name


*Setting Per-Connection / Sub-Engine Tokens:*

  logging name is not flexible enough for the case of tracking individual connections and/or transactions in log messages.
  
  Connection.*execution_options.logging_token* may be used to establish per-connection tracking tokens

*Hiding Parameters:*

  prevent these parameters from being logged for privacy purposes, enable the create_engine.hide_parameters flag

In [8]:
# Setting Per-Connection / Sub-Engine Tokens

from sqlalchemy import create_engine
e = create_engine("sqlite://", echo="debug")
with e.connect().execution_options(logging_token="track1") as conn:
    conn.execute("select 1").all()


e1 = e.execution_options(logging_token="track1")
e2 = e.execution_options(logging_token="track2")
with e1.connect() as conn:
    conn.execute("select 1").all()

with e2.connect() as conn:
    conn.execute("select 2").all()


# Hiding Parameters:
e = create_engine("sqlite://", echo=True, hide_parameters=True)
with e.connect() as conn:
    conn.execute(text("select :some_private_name"), {"some_private_name": "pii"})


2023-09-02 13:31:04,442 DEBUG sqlalchemy.pool.impl.SingletonThreadPool Created new connection <sqlite3.Connection object at 0x7f593d89f340>


DEBUG:sqlalchemy.pool.impl.SingletonThreadPool:Created new connection <sqlite3.Connection object at 0x7f593d89f340>


2023-09-02 13:31:04,445 DEBUG sqlalchemy.pool.impl.SingletonThreadPool Connection <sqlite3.Connection object at 0x7f593d89f340> checked out from pool


DEBUG:sqlalchemy.pool.impl.SingletonThreadPool:Connection <sqlite3.Connection object at 0x7f593d89f340> checked out from pool


2023-09-02 13:31:04,446 DEBUG sqlalchemy.pool.impl.SingletonThreadPool Connection <sqlite3.Connection object at 0x7f593d89f340> being returned to pool


DEBUG:sqlalchemy.pool.impl.SingletonThreadPool:Connection <sqlite3.Connection object at 0x7f593d89f340> being returned to pool


2023-09-02 13:31:04,448 DEBUG sqlalchemy.pool.impl.SingletonThreadPool Connection <sqlite3.Connection object at 0x7f593d89f340> rollback-on-return


DEBUG:sqlalchemy.pool.impl.SingletonThreadPool:Connection <sqlite3.Connection object at 0x7f593d89f340> rollback-on-return


ObjectNotExecutableError: Not an executable object: 'select 1'